In [1]:
using StatsBase
using Combinatorics
using Plots

include("jl/omega.jl")
include("jl/HSBM.jl")
# include("jl/inference.jl")

# parameters

n = 30
Z = rand(1:5, n)
ϑ = dropdims(ones(1,n) + rand(1,n), dims = 1)

# defining group intensity function Ω
μ = mean(ϑ)

kmax = 4
kmin = 1

fk = k->(2.4*μ*k)^(-k)
fp = harmonicMean

Ω_dict = Dict()

for k = kmin:kmax, p in partitions(k)
    Ω_dict[p] = fk(sum(p))*fp(p)
end

Ω = buildΩ(Ω_dict; by_size=true)

# sample from the HSBM with these parameters, restricting to hyperedges of size no more than kmax
H = sampleSBM(Z, ϑ, Ω; kmax=kmax, kmin = kmin)

hypergraph
  E: Dict{Integer,Dict}
  D: Array{Integer}((30,)) Integer[22, 23, 29, 22, 16, 23, 26, 36, 15, 26  …  32, 27, 24, 25, 24, 26, 17, 33, 29, 30]


In [8]:
# Create a random clustering with k clusters
include("jl/cut.jl")
k = 10
c = rand(1:k,n)
term1 = first_term_eval(H,c,Ω)

-1950.26642965325

In [9]:
# PC: broken under the dictionary-defined Ω. Could add these manually if desired, but not sure whether this is a problem or not. 

# Quick checks on the group evaluation function
# p4 = [4,0,0,0]
# p3 = [3,1,0,0]
# p2 = [2,2,0,0]
# p1 = [2,1,1,0]
# p0 = [1,1,1,1]
# @show Ω(p4;mode="partition"), Ω(p3;mode="partition"),Ω(p2;mode="partition"),Ω(p1;mode="partition"),Ω(p0;mode="partition")
# I'm going to think about some group interactions functions where the output is ordered
# the same as the way I've ordered here, i.e. Ω(p(i+1) >= Ω(p(i)
# Ordering by majorization seems natural for clustering applications

# PC: the harmonic mean will give you zeros whenever there is a group without any nodes in it. I'd suggest writing this informal experiment as in the next block: 

In [10]:
# still works because we haven't added zero-padding. 

for i = 1:4, p in partitions(4, i)
    print(p, " ")
    println(Ω(p;mode="partition"))
end

# an interesting note from this is that the harmonic mean doesn't actually respect the majorization order....

[4] 2.7505423676115396e-5
[3, 1] 2.5596680924654918e-5
[2, 2] 2.7505423676115396e-5
[2, 1, 1] 2.312921216928008e-5
[1, 1, 1, 1] 1.944927160079021e-5


In [25]:
@time T1 = first_term_eval(H,c,Ω)

  0.024509 seconds (5.38 k allocations: 15.973 MiB)


-1950.26642965325

In [26]:
# Here's a faster way to calculate it...for now requires a different way to store the hypergraph and Omega
ff = p->fp(p)*fk(sum(p))
# @time Om = build_omega(kmax,ff)

@time Hyp, w = hyperedge_formatting(H)

@time T2 = first_term_v2(Hyp,w,c,Ω)

@show T1-T2

  0.000086 seconds (23 allocations: 10.781 KiB)
  0.020317 seconds (5.37 k allocations: 15.973 MiB)
T1 - T2 = 3.183231456205249e-12


3.183231456205249e-12